**Libraries**

In [ ]:
pip install --upgrade category_encoders

     |████████████████████████████████| 81kB 3.5MB/s 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import category_encoders as ce

from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import CategoricalNB, BernoulliNB, MultinomialNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize


import plotly.graph_objects as go
from nltk.corpus import stopwords

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv("/content/drive/Shareddrives/Project 272/US_Accidents_Dec20.csv")

In [ ]:
new_df=df

In [ ]:
len(new_df)

4232541

In [ ]:
# new_df=new_df.iloc[:10000,:]
# new_df.to_csv("Road.csv")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_PATH = "/content/drive/Shareddrives/Project272"

In [ ]:
# df1=pd.read_csv('Road.csv')

In [ ]:
df.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Number,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,I-70 E,R,Dayton,Montgomery,OH,45424,US,US/Eastern,KFFO,2016-02-08 05:58:00,36.9,NaN,91.0,29.68,10.0,Calm,NaN,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,2584.0,Brice Rd,L,Reynoldsburg,Franklin,OH,43068-3402,US,US/Eastern,KCMH,2016-02-08 05:51:00,37.9,NaN,100.0,29.65,10.0,Calm,NaN,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,Accident on OH-32 State Route 32 Westbound at ...,NaN,State Route 32,R,Williamsburg,Clermont,OH,45176,US,US/Eastern,KI69,2016-02-08 06:56:00,36.0,33.3,100.0,29.67,10.0,SW,3.5,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,I-75 S,R,Dayton,Montgomery,OH,45417,US,US/Eastern,KDAY,2016-02-08 07:38:00,35.1,31.0,96.0,29.64,9.0,SW,4.6,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,NaN,Miamisburg Centerville Rd,R,Dayton,Montgomery,OH,45459,US,US/Eastern,KMGY,2016-02-08 07:53:00,36.0,33.3,89.0,29.65,6.0,SW,3.5,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,True,False,Day,Day,Day,Day


In [ ]:
len(new_df)

4232541

In [ ]:
pip install nltk

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import category_encoders as ce

from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import CategoricalNB, BernoulliNB, MultinomialNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize


import plotly.graph_objects as go
from nltk.corpus import stopwords

In [ ]:
road_features = ["Amenity", "Bump", "Crossing", "Give_Way", "Junction", "No_Exit", "Railway", "Roundabout", "Station", "Stop", "Traffic_Calming", "Traffic_Signal", "Turning_Loop"]
data = df[road_features].sum().sort_values(ascending=False)
data

Traffic_Signal     728111
Crossing           347417
Junction           339642
Station             86166
Stop                72183
Amenity             49430
Railway             38513
Give_Way            13195
No_Exit              5969
Traffic_Calming      2121
Bump                  861
Roundabout            237
Turning_Loop            0
dtype: int64

Preprocessing

In [ ]:
X=df
len(X)

4232541

In [ ]:
X.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Number,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,I-70 E,R,Dayton,Montgomery,OH,45424,US,US/Eastern,KFFO,2016-02-08 05:58:00,36.9,NaN,91.0,29.68,10.0,Calm,NaN,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,2584.0,Brice Rd,L,Reynoldsburg,Franklin,OH,43068-3402,US,US/Eastern,KCMH,2016-02-08 05:51:00,37.9,NaN,100.0,29.65,10.0,Calm,NaN,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,Accident on OH-32 State Route 32 Westbound at ...,NaN,State Route 32,R,Williamsburg,Clermont,OH,45176,US,US/Eastern,KI69,2016-02-08 06:56:00,36.0,33.3,100.0,29.67,10.0,SW,3.5,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,I-75 S,R,Dayton,Montgomery,OH,45417,US,US/Eastern,KDAY,2016-02-08 07:38:00,35.1,31.0,96.0,29.64,9.0,SW,4.6,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,NaN,Miamisburg Centerville Rd,R,Dayton,Montgomery,OH,45459,US,US/Eastern,KMGY,2016-02-08 07:53:00,36.0,33.3,89.0,29.65,6.0,SW,3.5,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,True,False,Day,Day,Day,Day


In [ ]:
X["Start_Time"] = pd.to_datetime(X["Start_Time"])

# Extract year, month, weekday and day
X["Year"] = X["Start_Time"].dt.year
X["Month"] = X["Start_Time"].dt.month
X["Weekday"] = X["Start_Time"].dt.weekday
X["Day"] = X["Start_Time"].dt.day

# Extract hour and minute
X["Hour"] = X["Start_Time"].dt.hour
X["Minute"] = X["Start_Time"].dt.minute

X.head(100)

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Number,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Year,Month,Weekday,Day,Hour,Minute
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,I-70 E,R,Dayton,Montgomery,OH,45424,US,US/Eastern,KFFO,2016-02-08 05:58:00,36.9,NaN,91.0,29.68,10.0,Calm,NaN,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night,2016,2,0,8,5,46
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,2584.0,Brice Rd,L,Reynoldsburg,Franklin,OH,43068-3402,US,US/Eastern,KCMH,2016-02-08 05:51:00,37.9,NaN,100.0,29.65,10.0,Calm,NaN,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Day,2016,2,0,8,6,7
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,Accident on OH-32 State Route 32 Westbound at ...,NaN,State Route 32,R,Williamsburg,Clermont,OH,45176,US,US/Eastern,KI69,2016-02-08 06:56:00,36.0,33.3,100.0,29.67,10.0,SW,3.5,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,True,False,Night,Night,Day,Day,2016,2,0,8,6,49
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,I-75 S,R,Dayton,Montgomery,OH,45417,US,US/Eastern,KDAY,2016-02-08 07:38:00,35.1,31.0,96.0,29.64,9.0,SW,4.6,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Day,Day,Day,2016,2,0,8,7,23
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,NaN,Miamisburg Centerville Rd,R,Dayton,Montgomery,OH,45459,US,US/Eastern,KMGY,2016-02-08 07:53:00,36.0,33.3,89.0,29.65,6.0,SW,3.5,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,True,False,Day,Day,Day,Day,2016,2,0,8,7,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,A-96,MapQuest,201.0,2,2016-02-11 07:28:16,2016-02-11 07:58:16,39.994061,-82.729416,NaN,NaN,0.01,Accident on OH-16 Broad St at County Hwy-41 Mi...,6686.0,Mink St SW,L,Pataskala,Licking,OH,43062-9413,US,US/Eastern,KVTA,2016-02-11 07:54:00,7.0,NaN,77.0,30.24,10.0,North,NaN,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day,2016,2,3,11,7,28
96,A-97,MapQuest,201.0,3,2016-02-11 07:29:06,2016-02-11 08:18:10,41.355396,-81.819267,NaN,NaN,0.01,Accident on I-71 Northbound at Exit 234 US-42 ...,NaN,I-71 S,R,Cleveland,Cuyahoga,OH,44130,US,US/Eastern,KCLE,2016-02-11 07:51:00,15.1,-1.6,51.0,30.18,10.0,NW,18.4,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day,2016,2,3,11,7,29
97,A-98,MapQuest,201.0,2,2016-02-11 07:53:55,2016-02-11 08:23:55,39.687389,-84.237968,NaN,NaN,0.00,Accident on Vance Rd at Main St. Expect delays.,NaN,Vance Rd,R,Dayton,Montgomery,OH,45439,US,US/Eastern,KMGY,2016-02-11 07:53:00,9.0,NaN,77.0,30.26,9.0,Calm,NaN,NaN,Clear,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day,2016,2,3,11,7,53
98,A-99,MapQuest,201.0,2,2016-02-11 08:07:02,2016-02

Start time to weekday,month,year

Dropping unwanted columns

In [ ]:
features_to_drop =  ['ID','Source','TMC','Start_Time','End_Time','End_Lat','End_Lng','Description','Number','Street',
                 'County', 'Zipcode','Country','Timezone','Airport_Code','Weather_Timestamp','Wind_Chill(F)',
                 'Turning_Loop','Sunrise_Sunset', 'Nautical_Twilight', 'Astronomical_Twilight','Distance(mi)',
                 'Side','Stop','Traffic_Signal','Civil_Twilight','Traffic_Calming','Amenity','Bump','Crossing',
                 'Give_Way','Junction','No_Exit','Railway','Roundabout','Station','City','State','Wind_Direction','Weather_Condition']
X = X.drop(features_to_drop, axis=1)
X.head()

,Severity,Start_Lat,Start_Lng,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Year,Month,Weekday,Day,Hour,Minute
0,3,39.865147,-84.058723,36.9,91.0,29.68,10.0,NaN,0.02,2016,2,0,8,5,46
1,2,39.928059,-82.831184,37.9,100.0,29.65,10.0,NaN,0.00,2016,2,0,8,6,7
2,2,39.063148,-84.032608,36.0,100.0,29.67,10.0,3.5,NaN,2016,2,0,8,6,49
3,3,39.747753,-84.205582,35.1,96.0,29.64,9.0,4.6,NaN,2016,2,0,8,7,23
4,2,39.627781,-84.188354,36.0,89.0,29.65,6.0,3.5,NaN,2016,2,0,8,7,39


In [ ]:
X.columns

Index(['Severity', 'Start_Lat', 'Start_Lng', 'Temperature(F)', 'Humidity(%)',
       'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Year', 'Month', 'Weekday', 'Day', 'Hour',
       'Minute'],
      dtype='object')

Dropping duplicate values

In [ ]:
print("Number of rows:", len(X.index))
X.drop_duplicates(inplace=True)
print("Number of rows after drop of duplicates:", len(X.index))

Number of rows: 4232541
Number of rows after drop of duplicates: 4047824


In [ ]:
X

,Severity,Start_Lat,Start_Lng,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Year,Month,Weekday,Day,Hour,Minute
0,3,39.865147,-84.058723,36.9,91.0,29.68,10.0,NaN,0.02,2016,2,0,8,5,46
1,2,39.928059,-82.831184,37.9,100.0,29.65,10.0,NaN,0.00,2016,2,0,8,6,7
2,2,39.063148,-84.032608,36.0,100.0,29.67,10.0,3.5,NaN,2016,2,0,8,6,49
3,3,39.747753,-84.205582,35.1,96.0,29.64,9.0,4.6,NaN,2016,2,0,8,7,23
4,2,39.627781,-84.188354,36.0,89.0,29.65,6.0,3.5,NaN,2016,2,0,8,7,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4232536,2,34.002480,-117.379360,86.0,40.0,28.92,10.0,13.0,0.00,2019,8,4,23,18,3
4232537,2,32.766960,-117.148060,70.0,73.0,29.39,10.0,6.0,0.00,2019,8,4,23,19,11
4232538,2,33.775450,-117.847790,73.0,64.0,29.74,10.0,10.0,0.00,2019,8,4,23,19,0
4232539,2,33.992460,-118.403020,71.0,81.0,29.62,10.0,8.0,0.00,2019,8,4,23,19,0


Handling missing values

In [ ]:
X[["Pressure(in)", "Visibility(mi)"]].describe().round(2)

,Pressure(in),Visibility(mi)
count,3976580.00,3955206.00
mean,29.70,9.11
std,0.86,2.83
min,0.00,0.00
25%,29.65,10.00
50%,29.93,10.00
75%,30.08,10.00
max,58.04,140.00


In [ ]:
X = X[X["Pressure(in)"] != 0]
X = X[X["Visibility(mi)"] != 0]
X[["Pressure(in)", "Visibility(mi)"]].describe().round(2)

,Pressure(in),Visibility(mi)
count,3974197.00,3952821.00
mean,29.70,9.12
std,0.85,2.82
min,0.02,0.06
25%,29.65,10.00
50%,29.93,10.00
75%,30.08,10.00
max,58.04,140.00


In [ ]:
# unique_weather = X["Weather_Condition"].unique()

# print(len(X["Weather_Condition"]))
# print(len(unique_weather))


Replacing Description for weather

In [ ]:

# print(unique_weather)

In [ ]:
# X.loc[X["Weather_Condition"].str.contains("Thunder|T-Storm", na=False), "Weather_Condition"] = "Thunderstorm"
# X.loc[X["Weather_Condition"].str.contains("Snow|Sleet|Wintry", na=False), "Weather_Condition"] = "Snow"
# X.loc[X["Weather_Condition"].str.contains("Rain|Drizzle|Shower", na=False), "Weather_Condition"] = "Rain"
# X.loc[X["Weather_Condition"].str.contains("Wind|Squalls", na=False), "Weather_Condition"] = "Windy"
# X.loc[X["Weather_Condition"].str.contains("Hail|Pellets", na=False), "Weather_Condition"] = "Hail"
# X.loc[X["Weather_Condition"].str.contains("Fair", na=False), "Weather_Condition"] = "Clear"
# X.loc[X["Weather_Condition"].str.contains("Cloud|Overcast", na=False), "Weather_Condition"] = "Clouds"
# X.loc[X["Weather_Condition"].str.contains("Mist|Haze|Fog", na=False), "Weather_Condition"] = "Fog"
# X.loc[X["Weather_Condition"].str.contains("Sand|Dust", na=False), "Weather_Condition"] = "Sand"
# X.loc[X["Weather_Condition"].str.contains("Smoke|Volcanic Ash", na=False), "Weather_Condition"] = "Smoke"
# X.loc[X["Weather_Condition"].str.contains("N/A Precipitation", na=False), "Weather_Condition"] = np.nan

# print(X["Weather_Condition"].unique())


changing description for wind direction

In [ ]:
# X["Wind_Direction"].unique()

In [ ]:
# X.loc[X["Wind_Direction"] == "CALM", "Wind_Direction"] = "Calm"
# X.loc[X["Wind_Direction"] == "VAR", "Wind_Direction"] = "Variable"
# X.loc[X["Wind_Direction"] == "East", "Wind_Direction"] = "E"
# X.loc[X["Wind_Direction"] == "North", "Wind_Direction"] = "N"
# X.loc[X["Wind_Direction"] == "South", "Wind_Direction"] = "S"
# X.loc[X["Wind_Direction"] == "West", "Wind_Direction"] = "W"

# X["Wind_Direction"] = X["Wind_Direction"].map(lambda x : x if len(x) != 3 else x[1:], na_action="ignore")

# X["Wind_Direction"].unique()


In [ ]:
X.head(10)

,Severity,Start_Lat,Start_Lng,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Year,Month,Weekday,Day,Hour,Minute
0,3,39.865147,-84.058723,36.9,91.0,29.68,10.0,NaN,0.02,2016,2,0,8,5,46
1,2,39.928059,-82.831184,37.9,100.0,29.65,10.0,NaN,0.00,2016,2,0,8,6,7
2,2,39.063148,-84.032608,36.0,100.0,29.67,10.0,3.5,NaN,2016,2,0,8,6,49
3,3,39.747753,-84.205582,35.1,96.0,29.64,9.0,4.6,NaN,2016,2,0,8,7,23
4,2,39.627781,-84.188354,36.0,89.0,29.65,6.0,3.5,NaN,2016,2,0,8,7,39
5,3,40.100590,-82.925194,37.9,97.0,29.63,7.0,3.5,0.03,2016,2,0,8,7,44
6,2,39.758274,-84.230507,34.0,100.0,29.66,7.0,3.5,NaN,2016,2,0,8,7,59
7,3,39.770382,-84.194901,34.0,100.0,29.66,7.0,3.5,NaN,2016,2,0,8,7,59
8,2,39.778061,-84.172005,33.3,99.0,29.67,5.0,1.2,NaN,2016,2,0,8,8,0
9,3,40.100590,-82.925194,37.4,100.0,29.62,3.0,4.6,0.02,2016,2,0,8,8,10


Analyzing missing values

In [ ]:
X.isna().sum()

Severity                   0
Start_Lat                  0
Start_Lng                  0
Temperature(F)         83729
Humidity(%)            88870
Pressure(in)           71241
Visibility(mi)         92617
Wind_Speed(mph)       470788
Precipitation(in)    2044600
Year                       0
Month                      0
Weekday                    0
Day                        0
Hour                       0
Minute                     0
dtype: int64

In [ ]:
features_to_fill = ["Temperature(F)", "Humidity(%)", "Pressure(in)", "Visibility(mi)", "Wind_Speed(mph)", "Precipitation(in)"]
X[features_to_fill] = X[features_to_fill].fillna(X[features_to_fill].mean())

X.dropna(inplace=True)

X.isna().sum()

Severity             0
Start_Lat            0
Start_Lng            0
Temperature(F)       0
Humidity(%)          0
Pressure(in)         0
Visibility(mi)       0
Wind_Speed(mph)      0
Precipitation(in)    0
Year                 0
Month                0
Weekday              0
Day                  0
Hour                 0
Minute               0
dtype: int64

In [ ]:
len(X)

4045438

In [ ]:
X.describe().round(2)

,Severity,Start_Lat,Start_Lng,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Year,Month,Weekday,Day,Hour,Minute
count,4045438.00,4045438.00,4045438.00,4045438.00,4045438.00,4045438.00,4045438.00,4045438.00,4045438.00,4045438.00,4045438.00,4045438.00,4045438.00,4045438.00,4045438.00
mean,2.32,36.42,-95.52,61.68,65.61,29.70,9.12,7.95,0.01,2018.39,7.08,2.46,15.86,12.13,29.98
std,0.54,4.94,17.31,18.36,22.47,0.85,2.79,4.99,0.12,1.32,3.55,1.75,8.71,5.39,17.29
min,1.00,24.56,-124.62,-89.00,1.00,0.02,0.06,0.00,0.00,2016.00,1.00,0.00,1.00,0.00,0.00
25%,2.00,33.54,-117.35,50.00,50.00,29.67,10.00,5.00,0.00,2017.00,4.00,1.00,8.00,8.00,15.00
50%,2.00,35.82,-90.10,63.00,67.00,29.93,10.00,7.95,0.01,2019.00,8.00,2.00,16.00,12.00,30.00
75%,3.00,40.19,-80.88,75.00,84.00,30.08,10.00,10.00,0.01,2020.00,10.00,4.00,23.00,17.00,45.00
max,4.00,49.00,-67.11,203.00,100.00,58.04,140.00,984.00,25.00,2020.00,12.00,6.00,31.00,23.00,59.00


Handling unbalanced data (Severity)

In [ ]:
size = len(X[X["Severity"]==1].index)
df = pd.DataFrame()
for i in range(1,5):
    S = X[X["Severity"]==i]
    df = df.append(S.sample(size, random_state=42))
X = df

In [ ]:
scaler = MinMaxScaler()
features = ['Temperature(F)','Humidity(%)','Pressure(in)','Visibility(mi)','Wind_Speed(mph)','Precipitation(in)','Start_Lng','Start_Lat','Year', 'Month','Weekday','Day','Hour','Minute']
X[features] = scaler.fit_transform(X[features])
X.head()

,Severity,Start_Lat,Start_Lng,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Year,Month,Weekday,Day,Hour,Minute
3655828,1,0.374356,0.217760,0.890078,0.102041,0.753006,0.132639,0.007292,0.0,1.0,0.272727,1.000000,0.833333,0.608696,0.186441
3614858,1,0.186694,0.762330,0.802529,0.612245,0.920444,0.132639,0.015800,0.0,1.0,0.272727,0.000000,0.166667,0.521739,0.983051
3443709,1,0.373760,0.710306,0.826848,0.612245,0.814986,0.132639,0.010938,0.0,1.0,0.363636,0.500000,0.900000,0.608696,0.372881
3472202,1,0.364244,0.220509,0.928988,0.040816,0.785384,0.132639,0.008508,0.0,1.0,0.363636,0.500000,0.200000,0.565217,0.796610
3464634,1,0.542588,0.040093,0.734436,0.642857,0.918594,0.132639,0.015800,0.0,1.0,0.363636,0.666667,0.000000,0.869565,0.101695


In [ ]:

import joblib

In [ ]:
scaler_filename = "saved_scaler"
joblib.dump(scaler, scaler_filename)

['saved_scaler']

Replacing True/False with 1/0

In [ ]:
X = X.replace([True, False], [1, 0])

X.head()

,Severity,Start_Lat,Start_Lng,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Year,Month,Weekday,Day,Hour,Minute
3655828,1,0.374356,0.217760,0.890078,0.102041,0.753006,0.132639,0.007292,0.0,1.0,0.272727,1.000000,0.833333,0.608696,0.186441
3614858,1,0.186694,0.762330,0.802529,0.612245,0.920444,0.132639,0.015800,0.0,1.0,0.272727,0.000000,0.166667,0.521739,0.983051
3443709,1,0.373760,0.710306,0.826848,0.612245,0.814986,0.132639,0.010938,0.0,1.0,0.363636,0.500000,0.900000,0.608696,0.372881
3472202,1,0.364244,0.220509,0.928988,0.040816,0.785384,0.132639,0.008508,0.0,1.0,0.363636,0.500000,0.200000,0.565217,0.796610
3464634,1,0.542588,0.040093,0.734436,0.642857,0.918594,0.132639,0.015800,0.0,1.0,0.363636,0.666667,0.000000,0.869565,0.101695


In [ ]:
X

,Severity,Start_Lat,Start_Lng,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Year,Month,Weekday,Day,Hour,Minute
3655828,1,0.374356,0.217760,0.890078,0.102041,0.753006,0.132639,0.007292,0.000000,1.00,0.272727,1.000000,0.833333,0.608696,0.186441
3614858,1,0.186694,0.762330,0.802529,0.612245,0.920444,0.132639,0.015800,0.000000,1.00,0.272727,0.000000,0.166667,0.521739,0.983051
3443709,1,0.373760,0.710306,0.826848,0.612245,0.814986,0.132639,0.010938,0.000000,1.00,0.363636,0.500000,0.900000,0.608696,0.372881
3472202,1,0.364244,0.220509,0.928988,0.040816,0.785384,0.132639,0.008508,0.000000,1.00,0.363636,0.500000,0.200000,0.565217,0.796610
3464634,1,0.542588,0.040093,0.734436,0.642857,0.918594,0.132639,0.015800,0.000000,1.00,0.363636,0.666667,0.000000,0.869565,0.101695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3904791,4,0.614872,0.820457,0.564202,0.683673,0.925994,0.132639,0.005591,0.001306,0.75,0.090909,1.000000,0.533333,0.304348,0.576271
4082727,4,0.340598,0.129632,0.676070,0.704082,0.917669,0.132639,0.009661,0.001306,0.50,0.000000,0.333333,0.066667,0.173913,0.118644
4019371,4,0.366697,0.217537,0.987840,0.102041,0.882516,0.132639,0.018230,0.001306,0.50,0.545455,0.000000,0.733333,0.695652,0.355932
4072787,4,0.741374,0.703093,0.522860,0.785714,0.932470,0.009875,0.007049,0.001306,0.50,0.090909,0.666667,0.266667,0.391304,0.711864


In [ ]:
# Metrics dictionary
accuracy = dict()
precision = dict()
recall = dict()
f1 = dict()
fpr = dict()
tpr = dict()

Train and validation

In [ ]:
# Train/Validation - Test split
X, X_test = train_test_split(X, test_size=.2, random_state=42)
print(X.shape, X_test.shape)

(84681, 15) (21171, 15)


In [ ]:
sample = X
y_sample = sample["Severity"]
X_sample = sample.drop("Severity", axis=1)

	
X_train, X_validate, y_train, y_validate = train_test_split(X_sample, y_sample, random_state=42)
print(X_train.shape, y_train.shape)
print(X_validate.shape, y_validate.shape)

(63510, 14) (63510,)
(21171, 14) (21171,)


In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import GradientBoostingClassifier
# from xgboost import XGBClassifier
# lrcv = LogisticRegression(max_iter = 500)
# gbc = GradientBoostingClassifier(learning_rate= 0.01,n_estimators= 200)
# xgb = XGBClassifier()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sample,y_sample, train_size = 0.75, random_state = 12)

In [ ]:
X_sample

,Start_Lat,Start_Lng,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Year,Month,Weekday,Day,Hour,Minute
4100434,0.630932,0.727332,0.535992,0.622449,0.913043,0.132639,0.005591,0.001306,0.25,1.000000,0.666667,0.233333,0.826087,0.322034
1339541,0.709575,0.911519,0.783074,0.836735,0.863090,0.079263,0.000000,0.000000,0.75,0.454545,0.500000,0.166667,0.304348,0.864407
3530413,0.291992,0.320001,0.831712,0.265306,0.545791,0.132639,0.018230,0.001306,1.00,0.454545,0.000000,0.233333,0.434783,0.813559
3833382,0.519363,0.044627,0.763619,0.438776,0.907493,0.105951,0.007292,0.000000,0.75,0.909091,1.000000,0.300000,0.565217,0.440678
2167819,0.428735,0.760668,0.763619,0.673469,0.891767,0.132639,0.015435,0.001306,0.50,0.272727,0.333333,0.100000,0.434783,0.389831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004135,0.539366,0.052047,0.671206,0.897959,0.913969,0.092607,0.010938,0.000000,0.75,1.000000,0.666667,0.166667,0.304348,0.016949
2634441,0.087952,0.780536,0.856518,0.714286,0.899167,0.132639,0.013977,0.001306,0.25,0.727273,0.000000,0.800000,0.608696,0.508475
4050105,0.360960,0.221294,0.797665,0.051020,0.907493,0.132639,0.009844,0.001306,0.50,0.272727,0.166667,0.533333,0.652174,0.457627
3538527,0.834541,0.026398,0.783074,0.469388,0.890842,0.132639,0.008508,0.000000,1.00,0.454545,0.500000,0.333333,0.782609,0.237288


In [ ]:
rfc = RandomForestClassifier(n_jobs=-1, random_state=42)
parameters = [{"n_estimators": [50, 100, 200, 500], "max_depth": [5, 10, 15, 30]}]
grid = GridSearchCV(rfc, parameters, verbose=5, n_jobs=-1)
grid.fit(X_train, y_train)

print("Best parameters scores:")
print(grid.best_params_)
print("Train score:", grid.score(X_train, y_train))
print("Validation score:", grid.score(X_validate, y_validate))

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   54.1s
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.

[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed: 12.4min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed: 18.5min finished


Best parameters scores:
{'max_depth': 30, 'n_estimators': 500}
Train score: 0.999921272240592
Validation score: 0.9146946294459402


In [ ]:
print("Default scores:")
rfc.fit(X_train, y_train)
print("Train score:", rfc.score(X_train, y_train))
print("Validation score:", rfc.score(X_validate, y_validate))

Default scores:
Train score: 0.9999527633443552
Validation score: 0.9141750507770063


In [ ]:
y_pred=rfc.predict(X_sample)

In [ ]:
import pickle
pickle.dump(rfc,open('lr_model.pkl','wb'))

In [ ]:
# infile = open(DATA_PATH+'/lr_model.pkl','rb')
# best_model2 = pickle.load(infile)

In [ ]:
y_sample

4100434    4
1339541    2
3530413    1
3833382    2
2167819    2
          ..
1004135    3
2634441    3
4050105    4
3538527    1
3507712    1
Name: Severity, Length: 84681, dtype: int64

In [ ]:
y_pred

array([4, 2, 1, ..., 4, 1, 1])

In [ ]:
print("Accuracy: ", accuracy_score(y_sample,y_pred))

Accuracy:  0.9142664824458852
